# Musical Instrument Classifier Training
Created by Arda Erol

In [ ]:
from fastai.vision.all import *
from pathlib import Path

### Mount Google Drive and Extract Dataset

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# !unzip -q '/content/drive/MyDrive/music_instruments.zip' -d /content
path = Path('/content/music_instruments')

### Preview Data and Folder Structure

In [ ]:
get_image_files(path)[:5]

### Create DataBlock and Loaders

In [ ]:
dblock = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    get_y=parent_label,
    item_tfms=Resize(460),
    batch_tfms=aug_transforms(size=224)
)
dls = dblock.dataloaders(path, bs=64)

### Show Sample Batch

In [ ]:
dls.show_batch(max_n=9, figsize=(8,6))

### Model and Learning Rate Finder

In [ ]:
learn = vision_learner(dls, resnet18, metrics=accuracy)
lr_min, lr_steep = learn.lr_find(suggest_funcs=(minimum, steep))
print(f"Suggested learning rates: min={lr_min}, steep={lr_steep}")

### Train the Model

In [ ]:
learn.fine_tune(3, base_lr=lr_min)

### Interpret Results

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(6,6))

### Save the Model

In [ ]:
learn.path = Path('.')
learn.export('music_instruments_model.pkl')
learn.save('music_weights')

### Download the Model File

In [ ]:
from google.colab import files
files.download('models/music_weights.pth')